In [1]:
import duckdb
from pathlib import Path
from dotenv import load_dotenv
import os

In [2]:
dotenv_path = Path("/Users/arthurgonzaga/DataKick/.env")
load_dotenv(dotenv_path=dotenv_path)

True

In [3]:

# Configuração das credenciais do MinIO
MINIO_ACCESS_KEY = os.getenv('MINIO_ACCESS_KEY')
MINIO_SECRET_KEY = os.getenv('MINIO_SECRET_KEY')
MINIO_ENDPOINT = "localhost:9000"

In [4]:
# Conectando ao DuckDB
con = duckdb.connect()

config = f"""
INSTALL httpfs;
LOAD httpfs;
SET s3_access_key_id='{MINIO_ACCESS_KEY}';
SET s3_secret_access_key='{MINIO_SECRET_KEY}';
SET s3_endpoint='{MINIO_ENDPOINT}'; -- Endpoint do MinIO
SET s3_url_style = 'path';
SET s3_use_ssl=false; -- Defina como 'true' se estiver usando SSL/TLS no MinIO
"""

# Configurando as credenciais e o endpoint do MinIO no DuckDB
con.execute(config)

In [5]:
print(config)


INSTALL httpfs;
LOAD httpfs;
SET s3_access_key_id='jknVRF5166kyQRLiK7gc';
SET s3_secret_access_key='OKczJuafJxBfAQ8xn9hiHguhbQtQgmYALDwLY1Sy';
SET s3_endpoint='localhost:9000'; -- Endpoint do MinIO
SET s3_url_style = 'path';
SET s3_use_ssl=false; -- Defina como 'true' se estiver usando SSL/TLS no MinIO



# Golden Set

In [6]:
minio_silver_path = "s3://silver/api_futebol/matches_stats.parquet"

In [62]:
query = f'''
WITH home_matches AS (
    SELECT 
        home_team AS team, 
        SUM(home_score) AS goals_for,
        SUM(away_score) AS goals_against,
        SUM(CAST(home_passes_completed AS INT)) AS passes_for_completed,
        SUM(CAST(home_passes_failed AS INT)) AS passes_for_failed,
        SUM(CAST(home_shoots_on_goal AS INT)) AS shoots_on_goal,
        SUM(CAST(home_shoots_off_goal AS INT)) AS shoots_off_goal,
        SUM(CAST(home_shoots_on_crossbar AS INT)) AS shoots_crossbar, 
        SUM(CAST(home_shoots_blocked AS INT)) AS shoots_blocked,
        COUNT(home_team) AS matches
    FROM 
        read_parquet('{minio_silver_path}')
    GROUP BY 
        home_team
),
away_matches AS (
    SELECT 
        away_team AS team, 
        SUM(away_score) AS goals_for,
        SUM(home_score) AS goals_against,
        SUM(CAST(away_passes_completed AS INT)) AS passes_for_completed,
        SUM(CAST(away_passes_failed AS INT)) AS passes_for_failed,
        SUM(CAST(away_shoots_on_goal AS INT)) AS shoots_on_goal,
        SUM(CAST(away_shoots_off_goal AS INT)) AS shoots_off_goal,
        SUM(CAST(away_shoots_on_crossbar AS INT)) AS shoots_crossbar, 
        SUM(CAST(away_shoots_blocked AS INT)) AS shoots_blocked,
        COUNT(away_team) AS matches
    FROM 
        read_parquet('{minio_silver_path}')
    GROUP BY 
        away_team
),
joined AS (
    SELECT 
        *, 
        'home' AS tag
    FROM 
        home_matches
    WHERE 
        team IS NOT NULL
    UNION ALL
    SELECT 
        *,
        'away' AS tag
    FROM 
        away_matches
    WHERE 
        team IS NOT NULL
)

SELECT 
    team, 
    SUM(goals_for) AS GF, 
    SUM(goals_against) AS GA,
    SUM(shoots_on_goal) AS sum_shoots_on_goal,
    ROUND(AVG(shoots_on_goal/matches),3) AS avg_shoots_on_goal_by_match,
    SUM(shoots_off_goal) AS sum_shoots_off_goal,
    ROUND(AVG(shoots_off_goal/matches),3) AS avg_shoots_off_goal_by_match,
    SUM(shoots_blocked) AS sum_shoots_blocked,
    ROUND(AVG(shoots_blocked/matches),3) AS avg_shoots_blocked_by_match,
    SUM(shoots_crossbar) AS sum_shoots_crossbar,
    ROUND(AVG(shoots_crossbar/matches),3) AS avg_shoots_crossbar_by_match,
    SUM(passes_for_completed) AS total_passes_completed,
    SUM(passes_for_failed) AS total_passes_failed,
    ROUND(AVG(passes_for_completed/matches),3) AS avg_passes_completed_by_match,
    ROUND(AVG(passes_for_failed/matches),3) AS avg_passes_failed_by_match,
    SUM(passes_for_completed)/(SUM(passes_for_completed)+SUM(passes_for_failed)) AS ratio_pass
FROM 
    joined
GROUP BY 
    team
;
'''

In [63]:
# Carregando o CSV do MinIO para uma tabela temporária e lendo os dados
try:
    result = con.execute(query).df()
except Exception as e:
    print("Erro ao acessar o MinIO:", e)

result

,team,GF,GA,sum_shoots_on_goal,avg_shoots_on_goal_by_match,sum_shoots_off_goal,avg_shoots_off_goal_by_match,sum_shoots_blocked,avg_shoots_blocked_by_match,sum_shoots_crossbar,avg_shoots_crossbar_by_match,total_passes_completed,total_passes_failed,avg_passes_completed_by_match,avg_passes_failed_by_match,ratio_pass
0,Fortaleza,31.0,24.0,88.0,3.487,119.0,4.750,66.0,2.612,6.0,0.234,6634.0,1804.0,264.199,72.147,0.786205
1,Flamengo,40.0,29.0,132.0,5.295,122.0,4.904,56.0,2.240,9.0,0.365,9457.0,1762.0,379.494,70.673,0.842945
2,São Paulo,33.0,26.0,102.0,4.096,99.0,3.955,59.0,2.381,12.0,0.478,10392.0,1554.0,416.423,62.314,0.869915
3,Atlético-GO,21.0,40.0,102.0,4.074,130.0,5.215,64.0,2.587,2.0,0.080,7097.0,1736.0,283.048,69.452,0.803464
4,Athletico-PR,27.0,29.0,117.0,4.712,137.0,5.535,73.0,2.933,12.0,0.494,8620.0,1870.0,345.859,74.740,0.821735
5,Botafogo,43.0,24.0,123.0,4.955,122.0,4.946,56.0,2.244,3.0,0.119,9087.0,1723.0,364.731,69.035,0.840611
6,Criciúma,34.0,36.0,104.0,4.154,126.0,5.035,74.0,2.936,6.0,0.247,7312.0,1773.0,293.401,70.891,0.804843
7,Corinthians,22.0,31.0,126.0,4.034,166.0,5.004,72.0,2.303,9.0,0.355,12844.0,1684.0,354.643,51.873,0.884086
8,Fluminense,21.0,26.0,99.0,3.939,99.0,3.974,42.0,1.667,6.0,0.237,10628.0,1653.0,425.022,66.228,0.865402
9,Atlético-MG,34.0,35.0,111.0,4.452,146.0,5.837,74.0,2.968,12.0,0.478,11334.0,1812.0,452.891,72.532,0.862163


In [46]:
query = f"SELECT SUM(CAST(home_shoots_on_goal AS BIGINT)) FROM read_parquet('{minio_silver_path}')"

In [10]:
result.columns

Index(['id_partida', 'championship_name', 'season', 'home_team', 'away_team',
       'home_score', 'away_score', 'status', 'dt_game_day',
       'home_ball_possession', 'home_corners', 'home_offside', 'home_passes',
       'home_passes_completed', 'home_passes_failed', 'home_shoots',
       'home_shoots_on_goal', 'home_shoots_off_goal',
       'home_shoots_on_crossbar', 'home_shoots_blocked', 'home_defenses',
       'home_disarms', 'home_tactical_schema', 'away_ball_possession',
       'away_corners', 'away_offside', 'away_passes', 'away_passes_completed',
       'away_passes_failed', 'away_shoots', 'away_shoots_on_goal',
       'away_shoots_off_goal', 'away_shoots_on_crossbar',
       'away_shoots_blocked', 'away_defenses', 'away_disarms',
       'away_tactical_schema'],
      dtype='object')

In [11]:
# Fechar a conexão
con.close()